In [13]:
# Env setup for notebook runs
from dotenv import load_dotenv
load_dotenv()
import os
print("APCA_API_KEY_ID set:", bool(os.getenv("APCA_API_KEY_ID")))
print("FINNHUB_API_KEY set:", bool(os.getenv("FINNHUB_API_KEY")))
print("SEC_USER_AGENT:", os.getenv("SEC_USER_AGENT", "<unset>"))


APCA_API_KEY_ID set: True
FINNHUB_API_KEY set: True
SEC_USER_AGENT: <unset>


In [17]:
import os
from dotenv import load_dotenv
from database.schema import TextChunks  # ensure this import is in a cell that's already run
load_dotenv()

# Set/override DB and API keys here if not in .env
os.environ.setdefault("DB_HOST", "localhost")
os.environ.setdefault("DB_PORT", "5432")
os.environ.setdefault("DB_NAME", "trading_agent")
os.environ.setdefault("DB_USER", "postgres")
os.environ.setdefault("DB_PASSWORD", "postgres")

# Required API keys
# os.environ["APCA_API_KEY_ID"] = "..."
# os.environ["APCA_API_SECRET_KEY"] = "..."
# os.environ["FRED_API_KEY"] = "..."
# os.environ["FINNHUB_API_KEY"] = "..."

print("DB:", os.getenv("DB_HOST"), os.getenv("DB_PORT"), os.getenv("DB_NAME"))
print("Alpaca keys present:", bool(os.getenv("APCA_API_KEY_ID") and os.getenv("APCA_API_SECRET_KEY")))
print("FRED key present:", bool(os.getenv("FRED_API_KEY")))
print("Finnhub key present:", bool(os.getenv("FINNHUB_API_KEY")))


ImportError: cannot import name 'TextChunks' from 'database.schema' (/Users/hakeemshindy/giga_hackathon/hackathon/database/schema.py)

In [15]:
from typing import List

# Dynamically fetch a large US symbol universe via Alpaca assets (approximate Russell 2000 coverage)
from data_sources.alpaca_client import AlpacaClient
assets_df = AlpacaClient().get_assets(asset_class="us_equity", status="active")
# Basic filter: keep uppercase tickers, length <=5, exclude obvious ETFs by quick heuristics
symbols_all = assets_df['symbol'].dropna().astype(str).str.upper().unique().tolist()
symbols_all = [s for s in symbols_all if s.isalnum() and len(s) <= 5]
# De-duplicate common ETF tickers (rough heuristic)
ETF_EXCLUDE = {"SPY","QQQ","IWM","DIA","VTI","VOO","XLK","XLF","XLE","XLV","XLY","XLP","XLI","XLU","IEMG","TLT","HYG","SCHX"}
symbols_all = [s for s in symbols_all if s not in ETF_EXCLUDE]

# Cap to ~2000 symbols
R2000_LIKE = symbols_all[:2000]

# Build batches for equities (100 per batch) and options (50 per batch)
EQUITY_SYMBOLS_BATCHES = [R2000_LIKE[i:i+100] for i in range(0, len(R2000_LIKE), 100)]
OPTIONS_UNDERLYINGS_BATCHES = [R2000_LIKE[i:i+50] for i in range(0, len(R2000_LIKE), 50)]

# News symbols can be the same universe, or reduced if desired
NEWS_SYMBOLS = R2000_LIKE

print("Universe size:", len(R2000_LIKE))
print("Equity batches:", len(EQUITY_SYMBOLS_BATCHES), "(100 each)")
print("Options batches:", len(OPTIONS_UNDERLYINGS_BATCHES), "(50 each)")


Universe size: 2000
Equity batches: 20 (100 each)
Options batches: 40 (50 each)


In [10]:
from database.storage import DataStorage
from database.schema import MarketData, OptionsContract, OptionsBar, SECFilings, MacroData, NewsArticle

def db_status():
    s = DataStorage()
    with s.get_session() as db:
        print("MarketData:", db.query(MarketData).count())
        print("OptionsContracts:", db.query(OptionsContract).count())
        print("OptionsBars:", db.query(OptionsBar).count())
        print("SECFilings:", db.query(SECFilings).count())
        print("MacroData:", db.query(MacroData).count())
        print("NewsArticles:", db.query(NewsArticle).count())

db_status()


MarketData: 338784
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098


In [11]:
from ingest import ingest_alpaca, ingest_news, ingest_alpaca_options
from datetime import datetime, timedelta

# Equities bars backfill by batches (large universe)
start_date = "2015-01-01"
end_date   = "2025-01-01"
for i, batch in enumerate(EQUITY_SYMBOLS_BATCHES, 1):
    print(f"Batch {i} equities {start_date}→{end_date} ({len(batch)} symbols)")
    try:
        ingest_alpaca(batch, start_date, end_date, timeframe="1Day")
    except Exception as e:
        print("Equities batch error:", e)
    db_status()

# Options bars backfill for underlyings batches (narrower batches)
opt_start_iso = "2025-06-01T00:00:00Z"
opt_end_iso   = "2025-06-10T00:00:00Z"
for i, underlyings in enumerate(OPTIONS_UNDERLYINGS_BATCHES[:20], 1):  # first ~1000 symbols
    print(f"Batch {i} options {opt_start_iso}→{opt_end_iso} ({len(underlyings)} symbols)")
    try:
        ingest_alpaca_options(underlyings, timeframe="15Min", start_iso=opt_start_iso, end_iso=opt_end_iso, exp_gte=None, exp_lte=None, max_contracts=500)
    except Exception as e:
        print("Options batch error:", e)
    db_status()

# News backfill in time windows to diversify coverage
news_windows = [
    ("2024-01-01T00:00:00Z", "2024-04-01T00:00:00Z"),
    ("2024-04-01T00:00:00Z", "2024-07-01T00:00:00Z"),
    ("2024-07-01T00:00:00Z", "2024-10-01T00:00:00Z"),
    ("2024-10-01T00:00:00Z", "2025-01-01T00:00:00Z"),
]
for (ws, we) in news_windows:
    print(f"News {ws}→{we} ({len(NEWS_SYMBOLS)} symbols)")
    try:
        ingest_news(NEWS_SYMBOLS, ws, we, use_finbert=False, max_articles=20000)
    except Exception as e:
        print("News window error:", e)
    db_status()


Batch 1 equities 2015-01-01→2025-01-01 (100 symbols)
✅ Ingested 56237 Alpaca bar records.
MarketData: 395021
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098
Batch 2 equities 2015-01-01→2025-01-01 (100 symbols)
✅ Ingested 54788 Alpaca bar records.
MarketData: 449809
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098
Batch 3 equities 2015-01-01→2025-01-01 (100 symbols)
✅ Ingested 54504 Alpaca bar records.
MarketData: 504313
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098
Batch 4 equities 2015-01-01→2025-01-01 (100 symbols)
✅ Ingested 44868 Alpaca bar records.
MarketData: 549181
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098
Batch 5 equities 2015-01-01→2025-01-01 (100 symbols)
✅ Ingested 42013 Alpaca bar records.
MarketData: 591194
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 53098


Error storing options contracts: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type json
LINE 1: ...can', 0, 'active', false, NULL, NULL, NULL, NULL, '{"id": "0...
                                                             ^
DETAIL:  Token "NaN" is invalid.
CONTEXT:  JSON data, line 1: ..."close_price_date": null, "prev_close_price": NaN...

[SQL: INSERT INTO options_contracts (symbol, root_symbol, underlying_symbol, expiration_date, option_type, strike_price, style, size, status, tradable, open_interest, open_interest_date, close_price, close_price_date, meta_data) VALUES (%(symbol)s, %(root_symbol)s, %(underlying_symbol)s, %(expiration_date)s, %(option_type)s, %(strike_price)s, %(style)s, %(size)s, %(status)s, %(tradable)s, %(open_interest)s, %(open_interest_date)s, %(close_price)s, %(close_price_date)s, %(meta_data)s::JSON) RETU

Options batch error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type json
LINE 1: ...can', 0, 'active', false, NULL, NULL, NULL, NULL, '{"id": "0...
                                                             ^
DETAIL:  Token "NaN" is invalid.
CONTEXT:  JSON data, line 1: ..."close_price_date": null, "prev_close_price": NaN...

[SQL: INSERT INTO options_contracts (symbol, root_symbol, underlying_symbol, expiration_date, option_type, strike_price, style, size, status, tradable, open_interest, open_interest_date, close_price, close_price_date, meta_data) VALUES (%(symbol)s, %(root_symbol)s, %(underlying_symbol)s, %(expiration_date)s, %(option_type)s, %(strike_price)s, %(style)s, %(size)s, %(status)s, %(tradable)s, %(open_interest)s, %(open_interest_date)s, %(close_price)s, %(close_price_date)s, %(meta_data)s::JSON) RETURNING option

In [7]:
# Optional: Hourly incremental loops (run for N hours) for news & equities
from time import sleep
from datetime import datetime, timedelta, timezone

HOURS = 3
WINDOW_DAYS_EQUITIES = 1
WINDOW_DAYS_NEWS = 7

for hour in range(HOURS):
    print(f"\n=== Hour {hour+1}/{HOURS} ===")
    # Equities: last day window
    try:
        from datetime import datetime, timedelta
        end_dt = datetime.now()
        start_dt = end_dt - timedelta(days=WINDOW_DAYS_EQUITIES)
        for i, batch in enumerate(EQUITY_SYMBOLS_BATCHES, 1):
            print(f"[Equities] Hourly batch {i}: {start_dt.date()}→{end_dt.date()} ({len(batch)} symbols)")
            ingest_alpaca(batch, start_dt.strftime('%Y-%m-%d'), end_dt.strftime('%Y-%m-%d'), timeframe="1Day")
    except Exception as e:
        print("Hourly equities error:", e)
    db_status()
    
    # News: last 7 days window
    try:
        end_iso = datetime.now(timezone.utc).isoformat()
        start_iso = (datetime.now(timezone.utc) - timedelta(days=WINDOW_DAYS_NEWS)).isoformat()
        print(f"[News] Hourly: {start_iso}→{end_iso} ({len(NEWS_SYMBOLS)} symbols)")
        ingest_news(NEWS_SYMBOLS, start_iso, end_iso, use_finbert=False, max_articles=10000)
    except Exception as e:
        print("Hourly news error:", e)
    db_status()

    # Sleep to next hour
    for rem in range(60):
        sleep(1)  # 60 minutes total




=== Hour 1/3 ===
[Equities] Hourly batch 1: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
[Equities] Hourly batch 2: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
[Equities] Hourly batch 3: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
MarketData: 338664
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 51358
[News] Hourly: 2025-08-07T05:28:43.267055+00:00→2025-08-14T05:28:43.267038+00:00 (60 symbols)
✅ Ingested 580 news articles.
MarketData: 338664
OptionsContracts: 100
OptionsBars: 6894
SECFilings: 6
MacroData: 59695
NewsArticles: 51938

=== Hour 2/3 ===
[Equities] Hourly batch 1: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
[Equities] Hourly batch 2: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
[Equities] Hourly batch 3: 2025-08-12→2025-08-13 (20 symbols)
✅ Ingested 20 Alpaca bar records.
MarketData: 338724
OptionsContracts: 100
Opt

In [26]:
# Deep DB status: include vectors
from sqlalchemy import func, text
from database.schema import TextChunks
from database.storage import DataStorage

s = DataStorage()
with s.get_session() as db:
    print("Counts → MarketData, OptionsContracts, OptionsBars, SECFilings, MacroData, NewsArticles, TextChunks")
    from database.schema import MarketData, OptionsContract, OptionsBar, SECFilings, MacroData, NewsArticle
    print(db.query(MarketData).count(), db.query(OptionsContract).count(), db.query(OptionsBar).count(), db.query(SECFilings).count(), db.query(MacroData).count(), db.query(NewsArticle).count(), db.query(TextChunks).count())
    try:
        by_src = dict(db.query(TextChunks.source, func.count()).group_by(TextChunks.source).all())
        print("TextChunks by source:", by_src)
        print("Latest TextChunk:", db.query(func.max(TextChunks.created_at)).scalar())
        print("Latest Filing:", db.query(func.max(SECFilings.filing_date)).scalar())
    except Exception as e:
        print("Agg error:", e)
    # Verify embeddings shape for last 5
    rows = db.query(TextChunks).order_by(TextChunks.created_at.desc()).limit(5).all()
    for r in rows:
        d = len(r.embedding) if r.embedding is not None else None
        print(f"{r.id} {r.source} {r.symbol} idx={r.chunk_index} emb_dim={d} doc={r.document_id[:60] if r.document_id else None}")



Counts → MarketData, OptionsContracts, OptionsBars, SECFilings, MacroData, NewsArticles, TextChunks
2480806 5884 7903 262 59695 91962 155935
TextChunks by source: {'news': 146545, 'sec': 9390}
Latest TextChunk: 2025-08-14 07:29:12.587399
Latest Filing: 2025-08-13 00:00:00
156127 news AAPL idx=0 emb_dim=384 doc=https://www.benzinga.com/general/education/25/01/42832123/wa
156126 news BTCUSD idx=0 emb_dim=384 doc=https://www.benzinga.com/general/macro-notification/25/01/42
156125 news AMD idx=0 emb_dim=384 doc=https://www.benzinga.com/25/01/42826372/intel-struggles-to-c
156124 news AAPL idx=0 emb_dim=384 doc=https://www.benzinga.com/trading-ideas/25/01/42824520/ai-qua
156123 news INTC idx=0 emb_dim=384 doc=https://www.benzinga.com/insights/options/25/01/42823833/10-


In [21]:
import importlib
import database.schema as schema
schema = importlib.reload(schema)
print("Loaded:", schema.__file__)
print("Has TextChunks:", hasattr(schema, "TextChunks"))
from database.schema import TextChunks  # should work now
from sqlalchemy import text
from database.storage import DataStorage

s = DataStorage()
with s.get_session() as db:
    print("DB:", db.bind.url.host, db.bind.url.port, db.bind.url.database)
    print("pgvector:", db.execute(text("select exists (select 1 from pg_extension where extname='vector')")).scalar())

Loaded: /Users/hakeemshindy/giga_hackathon/hackathon/database/schema.py
Has TextChunks: True
DB: localhost 5432 trading_agent
pgvector: True


In [29]:
from sqlalchemy import func
from database.schema import MarketData, OptionsContract, OptionsBar, SECFilings, MacroData, NewsArticle, TextChunks
from database.storage import DataStorage

s = DataStorage()
with s.get_session() as db:
    print("Counts:",
          "MarketData", db.query(MarketData).count(),
          "OptionsContracts", db.query(OptionsContract).count(),
          "OptionsBars", db.query(OptionsBar).count(),
          "SECFilings", db.query(SECFilings).count(),
          "MacroData", db.query(MacroData).count(),
          "NewsArticles", db.query(NewsArticle).count(),
          "TextChunks", db.query(TextChunks).count())
    print("TextChunks by source:", dict(db.query(TextChunks.source, func.count()).group_by(TextChunks.source).all()))
    print("Latest TextChunk:", db.query(func.max(TextChunks.created_at)).scalar())
    print("Latest Filing:", db.query(func.max(SECFilings.filing_date)).scalar())

Counts: MarketData 2480806 OptionsContracts 5884 OptionsBars 7903 SECFilings 262 MacroData 59695 NewsArticles 91962 TextChunks 155935
TextChunks by source: {'news': 146545, 'sec': 9390}
Latest TextChunk: 2025-08-14 07:29:12.587399
Latest Filing: 2025-08-13 00:00:00


In [23]:
from database.storage import DataStorage
from database.schema import TextChunks

s = DataStorage()
with s.get_session() as db:
    for r in db.query(TextChunks).order_by(TextChunks.created_at.desc()).limit(5):
        dim = len(r.embedding) if r.embedding is not None else None
        print(r.id, r.source, r.symbol, "idx", r.chunk_index, "emb_dim", dim, "doc", (r.document_id[:60] if r.document_id else None))

156127 news AAPL idx 0 emb_dim 384 doc https://www.benzinga.com/general/education/25/01/42832123/wa
156126 news BTCUSD idx 0 emb_dim 384 doc https://www.benzinga.com/general/macro-notification/25/01/42
156125 news AMD idx 0 emb_dim 384 doc https://www.benzinga.com/25/01/42826372/intel-struggles-to-c
156124 news AAPL idx 0 emb_dim 384 doc https://www.benzinga.com/trading-ideas/25/01/42824520/ai-qua
156123 news INTC idx 0 emb_dim 384 doc https://www.benzinga.com/insights/options/25/01/42823833/10-


In [27]:
from sqlalchemy import text
from database.storage import DataStorage

s = DataStorage()
with s.get_session() as db:
    row = db.execute(text("select id, document_id from text_chunks order by created_at desc limit 1")).fetchone()
    if row:
        print("Anchor chunk:", row[0], row[1])
        # kNN using vector distance; Postgres casts array-literal to vector
        # Use the stored embedding directly:
        q = text("""
          select id, source, document_id, chunk_index
          from text_chunks
          order by embedding <-> (select embedding from text_chunks where id=:id)
          limit 5
        """)
        print(db.execute(q, {"id": row[0]}).fetchall())
    else:
        print("No text_chunks found.")

Anchor chunk: 156127 https://www.benzinga.com/general/education/25/01/42832123/warren-buffett-beats-sp-500-in-2024-how-oracle-of-omahas-returns-compare-over-past-20-years
[(156127, 'news', 'https://www.benzinga.com/general/education/25/01/42832123/warren-buffett-beats-sp-500-in-2024-how-oracle-of-omahas-returns-compare-over-past-20-years', 0), (119375, 'news', 'https://www.benzinga.com/markets/24/06/39460971/warren-buffett-it-takes-just-a-few-winners-to-work-wonders', 0), (24912, 'news', 'https://www.benzinga.com/markets/24/06/39460971/warren-buffett-it-takes-just-a-few-winners-to-work-wonders', 0), (68704, 'news', 'https://www.benzinga.com/markets/24/06/39460971/warren-buffett-it-takes-just-a-few-winners-to-work-wonders', 0), (75018, 'news', 'https://www.benzinga.com/general/education/24/08/40317667/berkshire-hathaway-stock-up-19-so-far-in-2024-how-it-stacks-up-against-warren-buffetts-averages', 0)]


In [28]:
from sqlalchemy import text
from database.storage import DataStorage
s = DataStorage()
with s.get_session() as db:
    print("TextChunks:", db.execute(text("select count(*) from text_chunks")).scalar())
    print("By source:", db.execute(text("select source, count(*) from text_chunks group by source")).fetchall())
    print("Latest chunk:", db.execute(text("select max(created_at) from text_chunks")).scalar())

TextChunks: 155935
By source: [('news', 146545), ('sec', 9390)]
Latest chunk: 2025-08-14 07:29:12.587399
